## Imports

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

## Data Read-in

In [2]:
df = pd.read_csv("meatpacking_data.csv",index_col=0)
df2 = pd.read_csv("Retail Property Tracker - MeatpackingCoords.csv")
gdf = gpd.read_file("Meatpacking.kml")

## Data Clean

In [3]:
merged_df = pd.merge(left=df,left_on='bbl',right=df2,right_on='bbl',how='left')

In [4]:
merged_df = merged_df.rename(columns={'Vacant SF (From Offering Portfolio)':'Vacant SF'})

In [5]:
keep_columns = [
    'Address',
    'Retail Business (if vacant, put "vacant")',
    'ownername',
    'Vacant SF',
    'latitude',
    'longitude'
]

In [6]:
filtered_df = merged_df[keep_columns]

In [7]:
filtered_df = filtered_df[filtered_df['Address'] != "60 10th Ave"]
filtered_df = filtered_df[filtered_df['Vacant SF'] != "LEASED"]

In [8]:
filtered_df.loc[:, 'ownername'] = filtered_df['ownername'].str.replace(',', '')
# filtered_df.loc[:, 'Vacant SF'] = filtered_df['Vacant SF'].str.replace('-', '')
# filtered_df.loc[:, 'Vacant SF'] = filtered_df['Vacant SF'].str.replace('--', '')

In [9]:
geometry = [Point(xy) for xy in zip(filtered_df["longitude"], filtered_df["latitude"])]

In [10]:
points_gdf = gpd.GeoDataFrame(filtered_df, geometry=geometry, crs="EPSG:4326")

In [11]:
filtered_df.columns

Index(['Address', 'Retail Business (if vacant, put "vacant")', 'ownername',
       'Vacant SF', 'latitude', 'longitude'],
      dtype='object')

In [12]:
%store -r map_box_api_key

In [18]:
import folium
from branca.element import Template, MacroElement

# 1) Compute map center from the polygon’s centroid:
merged = gdf.geometry.union_all()
center = merged.centroid.coords[0]  # (lon, lat)

# 2) Create the Folium map first
m = folium.Map(
    location=[center[1], center[0]],  # folium wants [lat, lon]
    zoom_start=17
)

# 3) (Optional) add a Mapbox basemap layer if you want
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox",
    name="Streets (Mapbox)",
    overlay=False,
    control=False,
    show=True
).add_to(m)

# 4) Add the points from points_gdf
for _, row in points_gdf.iterrows():
    popup_text = f"<strong>{row.get('Address','')}</strong><br>Owner: {row.get('ownername','')}"
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=4,
        color="blue",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_text, max_width=400)
    ).add_to(m)

# 5) Now build and attach the legend
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px;
      left: 50px;
      width: 75px;
      height: 45px;
      background-color: white;
      border:2px solid grey;
      z-index:9999;
      font-size:14px;
      ">
    &nbsp;<strong>Legend</strong><br>
    &nbsp;<i style="
        background: blue;
        width: 10px;
        height: 10px;
        display: inline-block;
        margin-right: 8px;
        "></i>
      Vacant
  </div>
{% endmacro %}
"""

title_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      top: 10px;
      left: 50%;
      transform: translateX(-50%);
      background-color: white;
      padding: 6px 12px;
      border: 2px solid grey;
      z-index:9999;
      font-size:16px;
      font-weight: bold;
      ">
    Retail Vacancies in the Meatpacking District
  </div>
{% endmacro %}
"""

title = MacroElement()
title._template = Template(title_html)
m.get_root().add_child(title)

legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# 6) (Optional) if you want a layer control
folium.LayerControl().add_to(m)

# 7) Display inline in Jupyter
m


In [19]:
m.save('index.html')

In [17]:
import os

base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/meatpackingmap
